## Read gptw data (2022)

In [0]:
import pandas as pd
import numpy as np
import os

In [0]:
%run ./utils/user_defined_functions

## Utils functions used in notebooks

  Obtaining dependency information for Unidecode from https://files.pythonhosted.org/packages/8f/b7/559f59d57d18b44c6d1250d2eeaa676e028b9c527431f5d0736478a73ba1/Unidecode-1.4.0-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/235.8 kB ? eta -:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.8/235.8 kB 7.5 MB/s eta 0:00:00
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
# Lista KPI 
kpi_list = [
    "Number of Responses:",
    "Build more value together", "Camaraderie",
    "Caring", "Collaboration",
    "Communication", "Community",
    "Competence", "Corporate Image",
    "Credibility", "D&I Index",
    "Embrace the challenge and drive the change", "Engagement",
    "Equity", "Equity Index",
    "Fairness", "Hospitality",
    "Humble & Hungry", "Impartiality",
    "Innovation", "Integrity",
    "Intimacy", "Justice",
    "Keep it simple and do more with less", "Leadership Behavior",
    "Personal Job", "Pride",
    "Respect", "Respect other and the planet",
    "Support", "Team"
]


In [0]:
!pip install openpyxl
import pandas as pd
data = pd.read_excel(f"/Volumes/dev_advanced_analytics_hr/default/input_files/hr_turnover/input/gptw_survey/GPTW_2022.xlsx", skiprows=2, nrows=200)


  Obtaining dependency information for openpyxl from https://files.pythonhosted.org/packages/c0/da/977ded879c29cbd04de313843e76868e6e13408a94ed6b987245dc7c8506/openpyxl-3.1.5-py2.py3-none-any.whl.metadata
  Obtaining dependency information for et-xmlfile from https://files.pythonhosted.org/packages/c1/8b/5fe2cc11fee489817272089c4203e679c63b570a5aaeb18d852ae3cbba6a/et_xmlfile-2.0.0-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/250.9 kB ? eta -:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.9/250.9 kB 7.5 MB/s eta 0:00:00
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
data = data.rename(columns={
    "Campari Group\nGlobal Camparista Survey 2022\nOctober 10, 2022 - November 1, 2022\nFull Report - Global Results": "kpi"
})



In [0]:
data.columns = data.columns.to_series().apply(simplify_string)
data.drop(columns=["unnamed_0"], inplace = True)


In [0]:
data

,kpi,overall_nov_2022_,age,unnamed_4,unnamed_5,unnamed_6,unnamed_7,unnamed_8,assigned_team,unnamed_10,unnamed_11,unnamed_12,unnamed_13,unnamed_14,unnamed_15,unnamed_16,unnamed_17,unnamed_18,unnamed_19,unnamed_20,unnamed_21,unnamed_22,unnamed_23,unnamed_24,unnamed_25,unnamed_26,unnamed_27,unnamed_28,unnamed_29,unnamed_30,unnamed_31,unnamed_32,unnamed_33,unnamed_34,unnamed_35,unnamed_36,unnamed_37,unnamed_38,unnamed_39,unnamed_40,...,unnamed_1022,unnamed_1023,unnamed_1024,unnamed_1025,unnamed_1026,unnamed_1027,unnamed_1028,unnamed_1029,unnamed_1030,unnamed_1031,unnamed_1032,unnamed_1033,unnamed_1034,unnamed_1035,unnamed_1036,unnamed_1037,unnamed_1038,unnamed_1039,unnamed_1040,unnamed_1041,unnamed_1042,unnamed_1043,tier_9,unnamed_1045,unnamed_1046,unnamed_1047,unnamed_1048,unnamed_1049,unnamed_1050,type_of_job,unnamed_1052,unnamed_1053,unnamed_1054,unnamed_1055,work_status,unnamed_1057,unnamed_1058,zip_code,unnamed_1060,unnamed_1061
0,NaN,NaN,25 years or younger,26 years to 34 years,35 years to 44 years,45 years to 54 years,55 years or older,NaN,11.0,11000100.0,11000431.0,11000436.0,11000462.0,11000505.0,11000641.0,13000089.0,13000131.0,13000134.0,13000190.0,13000229.0,13000233.0,1330.0,1343.0,14000005.0,14000129.0,14000135.0,14000176.0,14000246.0,14000248.0,1504.0,16000006.0,16080003.0,18.0,18000183.0,18000234.0,18000279.0,18000325.0,18000388.0,1805.0,18080032.0,...,21000577.0,21000689.0,21000776.0,21000822.0,21000869.0,21000929.0,21002318.0,2136.0,30000946.0,30001734.0,30001852.0,30002349.0,30002513.0,30002643.0,30002809.0,4019.0,4045.0,6319.0,7000463.0,7000484.0,7000612.0,NaN,0.0,13000134.0,18000325.0,21000906.0,21001318.0,30002987.0,NaN,Executive,Managerial,"Staff, Professional, White Collars","Blue Collars, Hourly, Task Workers",NaN,Full-time,Part-time,NaN,40342.0,10036.0,10110.0
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Number of Responses:,3435.0,124,919,1208,726,290,NaN,7.0,6.0,11.0,7.0,6.0,6.0,7.0,11.0,8.0,5.0,5.0,6.0,9.0,6.0,9.0,19.0,8.0,5.0,5.0,9.0,6.0,6.0,10.0,6.0,5.0,8.0,6.0,13.0,5.0,10.0,5.0,8.0,...,17.0,10.0,7.0,5.0,9.0,10.0,12.0,5.0,7.0,6.0,5.0,10.0,6.0,5.0,6.0,6.0,5.0,7.0,5.0,5.0,6.0,NaN,2704.0,5.0,5.0,11.0,7.0,5.0,NaN,231,900,1377,824,NaN,3259,120,NaN,112.0,100.0,23.0
3,I am given the resources and equipment to do m...,74.0,81,75,74,75,78,NaN,43.0,100.0,91.0,100.0,50.0,100.0,100.0,100.0,100.0,100.0,80.0,100.0,78.0,83.0,56.0,63.0,88.0,80.0,80.0,89.0,67.0,83.0,90.0,100.0,100.0,75.0,83.0,62.0,60.0,90.0,80.0,50.0,...,82.0,60.0,29.0,40.0,11.0,90.0,75.0,60.0,86.0,83.0,100.0,90.0,83.0,20.0,83.0,50.0,40.0,43.0,80.0,60.0,67.0,NaN,77.0,100.0,60.0,91.0,57.0,60.0,NaN,85,74,76,70,NaN,75,69,NaN,63.0,70.0,61.0
4,This is a physically safe place to work.,88.0,80,86,91,90,88,NaN,100.0,100.0,82.0,100.0,50.0,100.0,86.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,80.0,100.0,83.0,100.0,90.0,100.0,100.0,100.0,100.0,92.0,40.0,100.0,100.0,75.0,...,35.0,90.0,57.0,80.0,44.0,80.0,58.0,100.0,86.0,100.0,100.0,80.0,100.0,80.0,83.0,83.0,60.0,86.0,80.0,100.0,100.0,NaN,93.0,100.0,40.0,18.0,86.0,100.0,NaN,94,96,89,77,NaN,89,75,NaN,82.0,96.0,96.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,People here are treated fairly regardless of t...,83.0,88,82,83,86,86,NaN,100.0,100.0,91.0,86.0,100.0,100.0,100.0,73.0,100.0,100.0,80.0,100.0,22.0,83.0,100.0,89.0,88.0,100.0,100.0,100.0,100.0,50.0,90.0,100.0,60.0,100.0,100.0,85.0,60.0,100.0,60.0,88.0,...,65.0,70.0,14.0,80.0,33.

In [0]:
def get_gptw_by_level(level_name, last_col_name):
    
    start_idx = data.columns.get_loc(level_name)
    end_idx = data.columns.get_loc(last_col_name) - 2
    df_level = data.iloc[:, [0] + list(range(start_idx, end_idx + 1))]

    # Imposta la prima riga come intestazione
    df_level.columns = df_level.iloc[0]
    df_level = df_level[1:].reset_index(drop=True)
    
    # Rinomina la prima colonna in "kpi"
    df_level.rename(columns={df_level.columns[0]: "kpi"}, inplace=True)
    df_level.columns = df_level.columns.fillna("temp_name")

    df_level.rename(columns={df_level.columns[0]: "kpi"}, inplace=True)

    # Filtra solo le KPI necessarie
    df_level = df_level[df_level["kpi"].isin(kpi_list)]

    # Applica la funzione simplify_string e aggiunge il prefisso "gptw_"
    df_level["kpi"] = df_level["kpi"].apply(simplify_string)
    df_level["kpi"] = "gptw_" + df_level["kpi"] 

    # Trasforma i dati in formato lungo 
    df_level = df_level.melt(id_vars=["kpi"], var_name="team_id", value_name="kpi_value")

    # Trasforma in pivot
    df_level = df_level.pivot(index="team_id", columns="kpi", values="kpi_value").reset_index()

    # Aggiunge la colonna merge_level e la sposta all'inizio
    #cols = ["merge_level"] + [col for col in df_level.columns if col != "merge_level"]
    #df_level = df_level[cols]
    
    
    df_level["merge_level"] = level_name
    df_level = df_level[["merge_level"] + [col for col in df_level.columns if col != "merge_level"]]

    return df_level


In [0]:
start_idx = data.columns.get_loc("assigned_team")
end_idx = data.columns.get_loc("birth_year") - 2
df_level = data.iloc[:, [0] + list(range(start_idx, end_idx + 1))]

# Imposta la prima riga come intestazione
df_level.columns = df_level.iloc[0]
df_level = df_level[1:].reset_index(drop=True)
df_level.rename(columns={df_level.columns[0]: "kpi"}, inplace=True)
df_level.columns = df_level.columns.fillna("temp_name")

# rename
df_level.rename(columns={df_level.columns[0]: "kpi"}, inplace=True)
df_level = df_level[df_level["kpi"].isin(kpi_list)]

# # Applica la funzione simplify_string e aggiunge il prefisso "gptw_"
df_level["kpi"] = df_level["kpi"].apply(simplify_string)
df_level["kpi"] = "gptw_" + df_level["kpi"] 


In [0]:
df_level

,kpi,11.0,11000100.0,11000431.0,11000436.0,11000462.0,11000505.0,11000641.0,13000089.0,13000131.0,13000134.0,13000190.0,13000229.0,13000233.0,1330.0,1343.0,14000005.0,14000129.0,14000135.0,14000176.0,14000246.0,14000248.0,1504.0,16000006.0,16080003.0,18.0,18000183.0,18000234.0,18000279.0,18000325.0,18000388.0,1805.0,18080032.0,1969.0,20000010.0,20000022.0,20000031.0,20000055.0,21000039.0,21000050.0,...,6710.0,6722.0,673.0,6751.0,6752.0,6777.0,6785.0,6789.0,6821.0,7000104.0,7000111.0,7000151.0,7000155.0,7000190.0,7000191.0,7000206.0,7000318.0,7000463.0,7000484.0,7000612.0,7000620.0,7000651.0,7000740.0,7000746.0,7000761.0,7000775.0,7000777.0,7000782.0,7000799.0,7000803.0,7000863.0,7080004.0,740.0,8.0,80007.0,80015.0,80032.0,80035.0,80044.0,80063.0
1,gptw_number_of_responses_,7.0,6.0,11.0,7.0,6.0,6.0,7.0,11.0,8.0,5.0,5.0,6.0,9.0,6.0,9.0,19.0,8.0,5.0,5.0,9.0,6.0,6.0,10.0,6.0,5.0,8.0,6.0,13.0,5.0,10.0,5.0,8.0,6.0,9.0,15.0,22.0,5.0,6.0,6.0,...,5.0,7.0,7.0,7.0,5.0,11.0,14.0,5.0,13.0,7.0,5.0,5.0,8.0,11.0,7.0,7.0,8.0,5.0,5.0,6.0,8.0,6.0,6.0,16.0,8.0,7.0,6.0,5.0,6.0,6.0,5.0,7.0,10.0,5.0,13.0,8.0,8.0,5.0,7.0,16.0
80,gptw_build_more_value_together,48.0,100.0,81.0,94.0,33.0,94.0,92.0,86.0,89.0,96.0,76.0,94.0,60.0,93.0,53.0,73.0,93.0,89.0,40.0,88.0,57.0,70.0,69.0,78.0,79.0,83.0,93.0,56.0,56.0,81.0,42.0,52.0,78.0,85.0,70.0,81.0,89.0,39.0,43.0,...,67.0,40.0,46.0,85.0,67.0,57.0,64.0,89.0,75.0,65.0,73.0,87.0,78.0,73.0,66.0,54.0,35.0,51.0,69.0,61.0,75.0,56.0,72.0,75.0,68.0,75.0,85.0,100.0,65.0,74.0,71.0,87.0,64.0,89.0,65.0,72.0,88.0,64.0,70.0,72.0
81,gptw_camaraderie,63.0,100.0,82.0,100.0,50.0,93.0,92.0,92.0,96.0,100.0,80.0,98.0,90.0,86.0,56.0,83.0,100.0,97.0,74.0,89.0,81.0,76.0,79.0,81.0,86.0,93.0,93.0,60.0,57.0,91.0,54.0,67.0,81.0,90.0,74.0,81.0,85.0,48.0,48.0,...,83.0,53.0,50.0,84.0,89.0,71.0,65.0,97.0,75.0,80.0,83.0,86.0,84.0,95.0,87.0,69.0,50.0,80.0,76.0,64.0,88.0,86.0,86.0,85.0,89.0,82.0,98.0,94.0,82.0,81.0,89.0,92.0,66.0,94.0,76.0,73.0,89.0,73.0,61.0,79.0
82,gptw_caring,59.0,98.0,75.0,98.0,36.0,95.0,88.0,83.0,86.0,100.0,74.0,95.0,56.0,79.0,83.0,81.0,100.0,91.0,69.0,87.0,62.0,74.0,66.0,88.0,74.0,86.0,93.0,67.0,46.0,80.0,50.0,66.0,69.0,79.0,75.0,77.0,82.0,50.0,29.0,...,80.0,57.0,76.0,88.0,63.0,60.0,70.0,97.0,75.0,82.0,94.0,91.0,82.0,84.0,78.0,80.0,48.0,77.0,83.0,69.0,50.0,83.0,83.0,78.0,75.0,96.0,98.0,97.0,83.0,81.0,71.0,92.0,73.0,91.0,75.0,76.0,84.0,79.0,63.0,82.0
83,gptw_collaboration,14.0,100.0,77.0,100.0,33.0,92.0,86.0,91.0,75.0,90.0,70.0,92.0,44.0,92.0,56.0,84.0,100.0,100.0,30.0,83.0,67.0,75.0,70.0,83.0,90.0,75.0,75.0,60.0,30.0,90.0,30.0,38.0,58.0,67.0,77.0,77.0,90.0,25.0,25.0,...,60.0,36.0,21.0,93.0,40.0,43.0,64.0,90.0,77.0,50.0,60.0,90.0,75.0,62.0,50.0,50.0,38.0,50.0,70.0,42.0,75.0,58.0,75.0,72.0,44.0,71.0,67.0,100.0,50.0,75.0,50.0,71.0,55.0,90.0,56.0,56.0,100.0,50.0,50.0,60.0
84,gptw_communication,50.0,100.0,77.0,89.0,38.0,100.0,89.0,82.0,88.0,90.0,60.0,100.0,64.0,88.0,81.0,88.0,97.0,85.0,30.0,94.0,62.0,67.0,68.0,75.0,65.0,84.0,83.0,65.0,65.0,85.0,50.0,53.0,54.0,78.0,73.0,81.0,80.0,50.0,46.0,...,90.0,46.0,36.0,89.0,45.0,64.0,73.0,90.0,73.0,71.0,75.0,90.0,69.0,80.0,82.0,39.0,47.0,65.0,70.0,62.0,78.0,42.0,75.0,73.0,44.0,64.0,88.0,90.0,58.0,75.0,65.0,75.0,75.0,100.0,65.0,53.0,91.0,65.0,50.0,78.0
85,gptw_community,57.0,100.0,64.0,100.0,50.0,100.0,71.0,91.0,88.0,100.0,80.0,100.0,67.0,100.0,33.0,74.0,100.0,80.0,60.0,78.0,67.0,67.0,60.0,67.0,80.0,100.0,100.0,46.0,60.0,80.0,60.0,71.0,100.0,100.0,67.0,77.0,80.0,50.0,83.0,...,60.0,29.0,86.0,86.0,100.0,55.0,64.0,80.0,69.0,57.0,80.0,80.0,88.0,91.0,100.0,57.0,25.0,40.0,80.0,50.0,62.0,33.0,50.0,81.0,75.0,71.0,83.0,100.0,83.0,50.0,80.0,100.0,60.0,100.0,58.0,75.0,100.0,80.0,57.0,80.0
86,gptw_competence,60.0,100.0,85.0,93.0,28.0,97.0,86.0,82.0,89.0,97.0,83.0,97.0,65.0,89.0,61.0,82.0,98.0,90.0,66.0,91.0,61.0,81.0,68.0,94.0,70.0,83.0,81.0,59.0,63.0,88.0,50.0,52.0,81.0,80.0,77.0,86.0,93.0,47.0,53.0,...,83.0,43.0,59.0,89.0,73.0,61.0,68.0,80.0,83.0,69.0,90.0,93.0,77.0,80.0,71.0,60.0,54.0,63.0,67.0,58.0,8

In [0]:
# Unisce i diversi livelli in un unico DataFrame
gptw_by_level = pd.concat([
    get_gptw_by_level('assigned_team', 'birth_year'),
    get_gptw_by_level('tier_1', 'tier_2'),
    get_gptw_by_level('tier_2', 'tier_3'),
    get_gptw_by_level('tier_3', 'tier_4'),
    get_gptw_by_level('tier_4', 'tier_5'),
    get_gptw_by_level('tier_5', 'tier_6'),
    get_gptw_by_level('tier_6', 'tier_7'),
    get_gptw_by_level('tier_7', 'tier_8'),
    get_gptw_by_level('tier_8', 'tier_9'),
    get_gptw_by_level('tier_9', 'type_of_job')
], ignore_index=True)


/home/spark-6e68260a-8d4f-4382-9898-4f/.ipykernel/1657/command-3332546666198271-222946114:28: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  df_level = df_level.pivot(index="team_id", columns="kpi", values="kpi_value").reset_index()
/home/spark-6e68260a-8d4f-4382-9898-4f/.ipykernel/1657/command-3332546666198271-222946114:28: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  df_level = df_level.pivot(index="team_id", columns="kpi", values="kpi_value").reset_index()
/home/spark-6e68260a-8d4f-4382-9898-4f/.ipykernel/1657/command-3332546666198271-222946114:28: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  df_level = df_level.pivot(index="team_id", columns="kpi", values="kpi_value").

In [0]:
# Converte team_id in numerico
gptw_by_level["team_id"] = pd.to_numeric(gptw_by_level["team_id"], errors='coerce')

# Rinomina la colonna gptw_number_of_responses_ in gptw_responders
gptw_by_level.rename(columns={"gptw_number_of_responses_": "gptw_responders"}, inplace=True)

In [0]:
# Load file excel
df_mapping = pd.read_excel(f"/Volumes/dev_advanced_analytics_hr/default/input_files/hr_turnover/input/gptw_survey/GPTW_2022_team_mapping.xlsx", skiprows=1) 
df_mapping.columns = df_mapping.columns.to_series().apply(simplify_string)

df_mapping = df_mapping[["user_employee_id", "manager_user_sys_id", "assigned_team"] + 
                        [col for col in df_mapping.columns if col.startswith("tier")]]

df_mapping["user_employee_id"] = pd.to_numeric(df_mapping["user_employee_id"], errors="coerce")
df_mapping["manager_user_sys_id"] = pd.to_numeric(df_mapping["manager_user_sys_id"], errors="coerce")

df_mapping = df_mapping.melt(id_vars=["user_employee_id", "manager_user_sys_id"], value_vars=[col for col in df_mapping.columns if "tier" in col] + ["assigned_team"], 
                              var_name="merge_level", value_name="team_id")
# team id different from 0
df_mapping = df_mapping[df_mapping["team_id"] != 0]

df_mapping = df_mapping.merge(gptw_by_level, how="left", on=["merge_level", "team_id"])

# Rendi 'assigned_team' come 'top' in 'merge_level' 
df_mapping["merge_level"] = df_mapping["merge_level"].apply(lambda x: "top" if x == "assigned_team" else x)
# Raggruppa per user_employee_id e ordina per merge_level in ordine decrescente
#df_mapping = df_mapping.sort_values(by=["user_employee_id", "merge_level"], ascending=[True, False])
df_mapping = df_mapping.groupby('user_employee_id').apply(lambda x: x.sort_values('merge_level', ascending=False)).reset_index(drop=True)
# Filtra per mantenere solo righe non con valori NA
df_mapping = df_mapping.dropna(how="any")

# Seleziona la prima riga per ciascun user_employee_id
df_mapping = df_mapping.groupby("user_employee_id").head(1)
# Ripristina 'top' come 'assigned_team'
df_mapping["merge_level"] = df_mapping["merge_level"].apply(lambda x: "assigned_team" if x == "top" else x)
df_mapping

,user_employee_id,manager_user_sys_id,merge_level,team_id,gptw_build_more_value_together,gptw_camaraderie,gptw_caring,gptw_collaboration,gptw_communication,gptw_community,gptw_competence,gptw_corporate_image,gptw_credibility,gptw_d_i_index,gptw_embrace_the_challenge_and_drive_the_change,gptw_engagement,gptw_equity,gptw_equity_index,gptw_fairness,gptw_hospitality,gptw_humble_hungry,gptw_impartiality,gptw_innovation,gptw_integrity,gptw_intimacy,gptw_justice,gptw_keep_it_simple_and_do_more_with_less,gptw_leadership_behavior,gptw_responders,gptw_personal_job,gptw_pride,gptw_respect,gptw_respect_other_and_the_planet,gptw_support,gptw_team
0,3,6785.0,assigned_team,6785,64.0,65.0,70.0,64.0,73.0,64.0,68.0,75.0,71.0,73.0,60.0,67.0,62.0,60.0,70.0,64.0,69.0,55.0,66.0,74.0,67.0,86.0,62.0,76.0,14.0,43.0,70.0,68.0,77.0,66.0,79.0
6,6,18.0,assigned_team,18,79.0,86.0,74.0,90.0,65.0,80.0,70.0,83.0,69.0,76.0,77.0,73.0,37.0,33.0,47.0,93.0,76.0,40.0,70.0,72.0,80.0,60.0,58.0,77.0,5.0,70.0,80.0,76.0,71.0,72.0,80.0
9,8,455.0,assigned_team,455,81.0,92.0,92.0,77.0,92.0,73.0,93.0,94.0,92.0,89.0,85.0,91.0,88.0,85.0,85.0,98.0,81.0,82.0,85.0,91.0,93.0,85.0,85.0,93.0,15.0,93.0,90.0,86.0,89.0,81.0,77.0
12,11,8.0,assigned_team,8,89.0,94.0,91.0,90.0,100.0,100.0,93.0,97.0,97.0,96.0,100.0,97.0,100.0,100.0,98.0,100.0,100.0,93.0,96.0,100.0,87.0,100.0,85.0,100.0,5.0,100.0,98.0,94.0,95.0,100.0,100.0
16,12,6558.0,assigned_team,6558,37.0,33.0,49.0,57.0,50.0,57.0,62.0,45.0,56.0,52.0,43.0,46.0,39.0,43.0,54.0,38.0,43.0,38.0,53.0,54.0,19.0,74.0,36.0,52.0,7.0,36.0,43.0,49.0,63.0,46.0,43.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26823,30005386,30005388.0,tier_7,30005383,70.0,88.0,74.0,75.0,67.0,67.0,78.0,78.0,71.0,85.0,86.0,74.0,25.0,20.0,51.0,94.0,60.0,39.0,63.0,67.0,89.0,80.0,75.0,72.0,6.0,75.0,80.0,67.0,67.0,53.0,89.0
26832,30005387,30005388.0,tier_7,30005383,70.0,88.0,74.0,75.0,67.0,67.0,78.0,78.0,71.0,85.0,86.0,74.0,25.0,20.0,51.0,94.0,60.0,39.0,63.0,67.0,89.0,80.0,75.0,72.0,6.0,75.0,80.0,67.0,67.0,53.0,89.0
26840,30005388,30005383.0,tier_7,30005383,70.0,88.0,74.0,75.0,67.0,67.0,78.0,78.0,71.0,85.0,86.0,74.0,25.0,20.0,51.0,94.0,60.0,39.0,63.0,67.0,89.0,80.0,75.0,72.0,6.0,75.0,80.0,67.0,67.0,53.0,89.0
26849,30005390,30005388.0,tier_7,30005383,70.0,88.0,74.0,75.0,67.0,67.0,78.0,78.0,71.0,85.0,86.0,74.0,25.0,20.0,51.0,94.0,60.0,39.0,63.0,67.0,89.0,80.0,75.0,72.0,6.0,75.0,80.0,67.0,67.0,53.0,89.0


In [0]:
df_mapping.to_csv(f"/Volumes/dev_advanced_analytics_hr/default/input_files/hr_turnover/data_cleaning/data_gptw_2022_v06.csv", sep=";", index=False) 